This notebook explores the flight data sample for the first 8 h of the first of jan 2024.

In [1]:
from __future__ import annotations

import polars as pl

from aia_model_contrail_avoidance.airports import list_of_uk_airports

In [2]:
parquet_file = "../../flight-data/2024_01_01_sample.parquet"
flight_dataframe = pl.read_parquet(parquet_file)
print(flight_dataframe.columns)

flight_dataframe.head()

['timestamp', 'source', 'callsign', 'icao_address', 'latitude', 'longitude', 'altitude_baro', 'altitude_gnss', 'on_ground', 'heading', 'speed', 'vertical_rate', 'squawk', 'aircraft_type_icao', 'tail_number', 'aircraft_typa_iata', 'aircraft_type name', 'airline_iata', 'airline_name', 'flight_number', 'departure_airport_icao', 'departure_airport_iata', 'arrival_airport_icao', 'arrival_airport_iata', 'departure_scheduled_time', 'arrival_scheduled_time', 'takeoff_time', 'landing_time', 'arrival_utc_offset', 'departure_utc_offset', 'aircraft_type_iata', 'aircraft_type_name', 'year', 'month', 'day', 'typecode', 'departure_airport_name', 'arrival_airport_name', 'flight_id']


timestamp,source,callsign,icao_address,latitude,longitude,altitude_baro,altitude_gnss,on_ground,heading,speed,vertical_rate,squawk,aircraft_type_icao,tail_number,aircraft_typa_iata,aircraft_type name,airline_iata,airline_name,flight_number,departure_airport_icao,departure_airport_iata,arrival_airport_icao,arrival_airport_iata,departure_scheduled_time,arrival_scheduled_time,takeoff_time,landing_time,arrival_utc_offset,departure_utc_offset,aircraft_type_iata,aircraft_type_name,year,month,day,typecode,departure_airport_name,arrival_airport_name,flight_id
"datetime[μs, UTC]",str,str,str,f64,f64,f64,f64,bool,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,datetime[μs],datetime[μs],datetime[μs],datetime[μs],f64,f64,str,str,i32,i8,i8,str,str,str,u32
2024-01-01 00:00:00 UTC,"""ADSB""","""TAM8085""","""e48ba7""",29.696417,-15.103455,34000.0,null,false,206.13857,470.0,0.0,4506.0,"""B77W""","""PT-MUF""",null,null,"""JJ""","""LATAM Brasil""","""LA8085""","""EGLL""","""LHR""","""SBGR""","""GRU""",2023-12-31 20:15:00,2024-01-01 08:00:00,2023-12-31 20:38:33,null,-1.08e7,0.0,null,"""Boeing 777-32WER""",2024,1,1,"""B77W""","""London Heathrow Airport""","""Guarulhos - Governador André F…",35227
2024-01-01 00:00:00 UTC,"""ADSB""","""TOM7BX""","""4076b1""",43.37581,-7.65185,38000.0,null,false,31.029434,null,0.0,6237.0,"""B738""","""G-TAWX""",null,null,"""BY""","""TUI Airways""","""BY4767""","""GCRR""","""ACE""","""EGKK""","""LGW""",2023-12-31 21:10:00,2024-01-01 01:15:00,null,null,0.0,0.0,null,"""Boeing 737-8K5""",2024,1,1,"""B738""","""Lanzarote Airport""","""London Gatwick Airport""",3650
2024-01-01 00:00:00 UTC,"""ADSB""","""DAL30""","""ab8c19""",36.899864,-79.39724,33000.0,null,false,47.92712,520.0,0.0,5743.0,"""B764""","""N843MH""",null,null,"""DL""","""Delta Air Lines""","""DL30""","""KATL""","""ATL""","""EGLL""","""LHR""",2023-12-31 23:05:00,2024-01-01 07:15:00,2023-12-31 23:17:09,null,0.0,-1.8e7,null,"""Boeing 767-432ER""",2024,1,1,"""B764""","""Hartsfield Jackson Atlanta Int…","""London Heathrow Airport""",32038
2024-01-01 00:00:00 UTC,"""ADSB""","""UAL929""","""a8a87e""",48.34456,-70.52589,35000.0,33775.0,false,73.33019,498.0,0.0,3156.0,"""B763""","""N657UA""",null,null,"""UA""","""United Airlines""","""UA929""","""KORD""","""ORD""","""EGLL""","""LHR""",2023-12-31 21:55:00,2024-01-01 05:55:00,2023-12-31 22:08:27,null,0.0,-2.16e7,null,"""Boeing 767-332ER""",2024,1,1,"""B763""","""Chicago O'Hare International A…","""London Heathrow Airport""",30010
2024-01-01 00:00:00 UTC,"""ADSB""","""EZY50FH""","""407687""",53.45314,-1.984406,4075.0,3525.0,false,336.41776,170.0,-672.0,3566.0,"""A20N""","""G-UZHS""",null,null,"""U2""","""easyJet Airline""","""U22060""","""LGAV""","""ATH""","""EGCC""","""MAN""",2023-12-31 19:40:00,2023-12-31 23:55:00,2023-12-31 20:25:21,null,0.0,7.2e6,null,"""Airbus A320-251N""",2024,1,1,"""A20N""","""Eleftherios Venizelos Internat…","""Manchester Airport""",3638


In [3]:
# number of flights
print(f"Number of flights: {flight_dataframe['flight_id'].n_unique()}")

# destination pairs count
dest_pairs = (
    flight_dataframe.group_by(["departure_airport_name", "arrival_airport_name"])
    .agg(pl.col("flight_id").n_unique().alias("num_flights"))
    .sort("num_flights", descending=True)
)
print("Top 10 destination pairs by number of flights:")
print(dest_pairs.head(10))

Number of flights: 1064
Top 10 destination pairs by number of flights:
shape: (10, 3)
┌─────────────────────────────────┬─────────────────────────┬─────────────┐
│ departure_airport_name          ┆ arrival_airport_name    ┆ num_flights │
│ ---                             ┆ ---                     ┆ ---         │
│ str                             ┆ str                     ┆ u32         │
╞═════════════════════════════════╪═════════════════════════╪═════════════╡
│ null                            ┆ null                    ┆ 201         │
│ Sangster International Airport  ┆ London Gatwick Airport  ┆ 21          │
│ Grantley Adams International A… ┆ Manchester Airport      ┆ 17          │
│ Hewanorra International Airpor… ┆ London Gatwick Airport  ┆ 16          │
│ Dubai International Airport     ┆ London Heathrow Airport ┆ 14          │
│ John F Kennedy International A… ┆ Manchester Airport      ┆ 12          │
│ Singapore Changi Airport        ┆ London Heathrow Airport ┆ 12          │
│ 

In [4]:
# number of unique arrival-departure pairs
print(f"Number of unique arrival-departure pairs: {dest_pairs.height}")
# number of pairs with only one flight
single_flight_pairs = dest_pairs.filter(pl.col("num_flights") == 1)
print(f"Number of arrival-departure pairs with only one flight: {single_flight_pairs.height}")

# number of flights to or from heathrow
heathrow_flights = flight_dataframe.filter(
    (pl.col("departure_airport_name") == "London Heathrow Airport")
    | (pl.col("arrival_airport_name") == "London Heathrow Airport")
)
print(f"Number of flights to or from London Heathrow: {heathrow_flights['flight_id'].n_unique()}")

Number of unique arrival-departure pairs: 634
Number of arrival-departure pairs with only one flight: 471
Number of flights to or from London Heathrow: 431
Number of flights to or from London Heathrow: 431


In [5]:
uk_airports = list_of_uk_airports()

# number of flights to and from UK airports
uk_flights = flight_dataframe.filter(
    (pl.col("departure_airport_icao").is_in(uk_airports))
    & (pl.col("arrival_airport_icao").is_in(uk_airports))
)
print(f"Number of flights to or from UK airports: {uk_flights['flight_id'].n_unique()}")

# destination pairs for UK flights
uk_dest_pairs = (
    uk_flights.group_by(["departure_airport_name", "arrival_airport_name"])
    .agg(pl.col("flight_id").n_unique().alias("num_flights"))
    .sort("num_flights", descending=True)
)
print("Top 10 UK destination pairs by number of flights:")
print(uk_dest_pairs.head(10))

Number of flights to or from UK airports: 55
Top 10 UK destination pairs by number of flights:
shape: (10, 3)
┌───────────────────────────────┬───────────────────────────────┬─────────────┐
│ departure_airport_name        ┆ arrival_airport_name          ┆ num_flights │
│ ---                           ┆ ---                           ┆ ---         │
│ str                           ┆ str                           ┆ u32         │
╞═══════════════════════════════╪═══════════════════════════════╪═════════════╡
│ London Stansted Airport       ┆ Belfast International Airport ┆ 2           │
│ Bristol Airport               ┆ Edinburgh Airport             ┆ 2           │
│ Manchester Airport            ┆ Belfast International Airport ┆ 2           │
│ Edinburgh Airport             ┆ London Gatwick Airport        ┆ 2           │
│ Glasgow International Airport ┆ London Gatwick Airport        ┆ 2           │
│ Belfast International Airport ┆ London Stansted Airport       ┆ 2           │
│ Glasgow 

In [6]:
from datetime import datetime

# show number of flgihts where the landing time is <8 am UTC
latest_timestamp = datetime(2024, 1, 1, 8, 0, 0)  # noqa: DTZ001
morning_landings = uk_flights.filter(pl.col("landing_time") < latest_timestamp)
print(f"Number of flights landing before 8 am: {morning_landings['flight_id'].n_unique()}")
list_of_uk_complete_flight_ids = morning_landings["flight_id"].unique().to_list()
print(f"List of flight IDs landing before 8 am: {list_of_uk_complete_flight_ids}")

Number of flights landing before 8 am: 31
List of flight IDs landing before 8 am: [3297, 3323, 3359, 3363, 3371, 3397, 3399, 3400, 3402, 3404, 3407, 3409, 3410, 3411, 3418, 3425, 3456, 3481, 3579, 3630, 3635, 3643, 3644, 3680, 3681, 3719, 3724, 3726, 3738, 3742, 3747]


In [7]:
# show the flight ids for each pair
for row in uk_dest_pairs.head(10).iter_rows(named=True):
    dep_airport = row["departure_airport_name"]
    arr_airport = row["arrival_airport_name"]
    flight_ids = (
        morning_landings.filter(
            (pl.col("departure_airport_name") == dep_airport)
            & (pl.col("arrival_airport_name") == arr_airport)
        )["flight_id"]
        .unique()
        .to_list()
    )
    print(f"Flights from {dep_airport} to {arr_airport}: {flight_ids}")

Flights from London Stansted Airport to Belfast International Airport: []
Flights from Bristol Airport to Edinburgh Airport: []
Flights from Manchester Airport to Belfast International Airport: [3399, 3719]
Flights from Edinburgh Airport to London Gatwick Airport: [3409, 3456]
Flights from Glasgow International Airport to London Gatwick Airport: [3742]
Flights from Belfast International Airport to London Stansted Airport: [3363, 3726]
Flights from Glasgow International Airport to London Heathrow Airport: [3297]
Flights from Newcastle Airport to Belfast International Airport: []
Flights from Inverness Airport to London Gatwick Airport: []
Flights from Belfast International Airport to Liverpool John Lennon Airport: []
